# EPH - Individuos


## Ajuste de ruta del proyecto en Notebook

In [1]:
import sys

# Ajusto para que la reconozca la ruta del proyecto
sys.path.append("..") 

## Unificación de los archivos de Individuos

In [2]:
from src.utils.constants import DATA_SOURCE_DIR,DATA_CLEAN_DIR,FILENAME_INDIVIDUOS_UNIFIED
from src.utils.helpers import  process_file, save_to_txt

# Procesar y guardar individuos
encabezados,individual = process_file(DATA_SOURCE_DIR, category="individual")
save_to_txt(encabezados,individual, DATA_CLEAN_DIR , FILENAME_INDIVIDUOS_UNIFIED)


✅ Archivo TXT guardado en: C:\Users\MONSO\code\data\clean\individuos_unificado.txt


## Procesamiento de los archivos

Abro el archivo unificado

In [3]:
from src.utils.constants import INDIVIDUOS_UNIFIED_DIR
from src.utils.helpers import read_file_dic

header,data=read_file_dic(INDIVIDUOS_UNIFIED_DIR)

Proceso el archivo

In [4]:
import src.procesamientos.individuos as indiv

indiv.add_extra_data(header,data)

Guardo los datos procesados a un archivo

In [5]:
from src.utils.constants import INDIVIDUOS_PROCESSED_DIR
from src.utils.helpers import save_to_csv

save_to_csv(INDIVIDUOS_PROCESSED_DIR,header,data, delimiter=";")

## Análisis de Individuos

Abro el archivo de individuos procesado:

In [6]:
from src.utils.helpers import read_file_dic

header,data=read_file_dic(INDIVIDUOS_PROCESSED_DIR)

Importo las funciones para realizar las consultas

In [7]:
import src.consultas.consultas as cs

In [8]:
import importlib

importlib.reload(cs)

<module 'src.consultas.consultas' from 'c:\\Users\\MONSO\\code\\notebooks\\..\\src\\consultas\\consultas.py'>

### 1- A partir de la información de cada año contenida en el dataset se debe informar, año tras año, el porcentaje de personas mayores a 6 años capaces e incapaces de leer y escribir.

In [9]:
cs.cant_personas_alfabetizadas(data)

Año           % Alfabetos      % No Alfabetos
---------------------------------------------
2024                98.28                1.72
2023                97.84                2.16


### 2- A partir de un año y trimestre elegido por el usuario informar el porcentaje de personas no nacidas en Argentina que hayan cursado un nivel universitario o superior

In [10]:
#Ingreso de datos por el usuario
ano=input("Ingrese el año: ")
trim=input("Ingrese el trimestre [1 a 4]: ")

cs.porc_extranjero_universitario(ano,trim,data)

El % de personas extranjeras que han cursado el nivel superior o universitario en el trimestre 3 del año 2024 es del: 5.92%


### 3- A partir de la información contenida en el dataset informar el año y trimestre donde hubo menor desocupación.

In [11]:
cs.info_menor_desocupacion(data)

Valor mínimo de desocupación: 822050 en los siguientes años y trimestres:
Año: 2023, Trimestre: 4


### 4- Ranking de los 5 aglomerados con mayor porcentaje de hogares con dos o más ocupantes con estudios universitarios o superiores finalizados. Información obtenida a partir del par de archivos más recientes. 

In [12]:
# 1. Cuento cuántos universitarios hay por hogar y guardar pondera

universitarios_por_hogar, pondera_por_hogar = cs.contar_universitarios_y_pondera_por_hogar(data)

# 2. Filtro hogares con al menos 2 universitarios

hogares_filtrados = cs.filtrar_hogares_con_min_universitarios(universitarios_por_hogar,pondera_por_hogar,min_universitarios=2)

In [13]:
# 3. Suma ponderada de todos los hogares

total_hogares = cs.contar_hogares(pondera_por_hogar)

# 4. Suma poderada de hogares con 2+ universitarios por aglomerado

hogares_con_universitarios_min = cs.contar_hogares(hogares_filtrados)

In [14]:
# 5. Calculo porcentajes  y ordeno por porcentajes
# También se imprime el nombre del aglomerado, el total de hogares y el total de hogares con al 
# menos 2 universitarios

top_5_aglomerados =cs.obtener_top_n_porcentaje_hogares_universitarios(total_hogares, hogares_con_universitarios_min )

Procesando aglomerado: 32, Total: 5080948.0, Universitarios: 680128.0
Procesando aglomerado: 33, Total: 16584842.0, Universitarios: 637504.0
Procesando aglomerado: 22, Total: 272773.0, Universitarios: 10245.0
Procesando aglomerado: 19, Total: 445718.0, Universitarios: 12076.0
Procesando aglomerado: 13, Total: 2403670.0, Universitarios: 106021.0
Procesando aglomerado: 36, Total: 287804.0, Universitarios: 17168.0
Procesando aglomerado: 14, Total: 210530.0, Universitarios: 3962.0
Procesando aglomerado: 15, Total: 331707.0, Universitarios: 9068.0
Procesando aglomerado: 8, Total: 558033.0, Universitarios: 19448.0
Procesando aglomerado: 9, Total: 332684.0, Universitarios: 9196.0
Procesando aglomerado: 25, Total: 287912.0, Universitarios: 7315.0
Procesando aglomerado: 91, Total: 229196.0, Universitarios: 5380.0
Procesando aglomerado: 93, Total: 128419.0, Universitarios: 3795.0
Procesando aglomerado: 34, Total: 1014559.0, Universitarios: 43914.0
Procesando aglomerado: 38, Total: 288180.0, Univ

In [15]:
# 7. Impresión del ranking de 5 aglomerados con mayor porcentaje de hogares con 2 o más universitarios
cs.imprimir_ranking_aglomerados(top_5_aglomerados, cantidad=5)

Ranking de los 5 aglomerados con mayor porcentaje de hogares con 2 o más universitarios:
1. Aglomerado 32 - Ciudad Autónoma de Buenos Aires: 13.39%
2. Aglomerado 2 - Gran La Plata: 7.50%
3. Aglomerado 10 - Gran Mendoza: 6.88%
4. Aglomerado 17 - Neuquén - Plottier: 6.66%
5. Aglomerado 6 - Gran Paraná: 6.36%


### 7- Informar para cada aglomerado el porcentaje de personas mayores que hayan cursado al menos en nivel universitario o superior.

Consideramos como muestra total para el cálculo porcentual sólo a las personas mayores de edad, que son las que tienen la posibilidad real de poder alcanzarlo.


In [16]:
cs.info_porcentual_educacionsuperior_aglomerado(data)

Aglomerado                               Porcentaje (%)
-------------------------------------------------------
32 - Ciudad Autónoma de Buenos Aires              62.99%
2 - Gran La Plata                                 45.70%
3 - Bahía Blanca - Cerri                          41.36%
13 - Gran Córdoba                                 40.95%
6 - Gran Paraná                                   40.19%
4 - Gran Rosario                                  39.29%
36 - Río Cuarto                                   39.25%
12 - Corrientes                                   39.01%
19 - Jujuy - Palpalá                              38.93%
34 - Mar del Plata                                38.63%
10 - Gran Mendoza                                 37.65%
23 - Gran Salta                                   37.58%
7 - Posadas                                       37.56%
17 - Neuquén - Plottier                           36.09%
25 - La Rioja                                     35.87%
29 - Gran Tucumán - Tafí Viejo   

### 9- Pedir al usuario que seleccione un aglomerado y a partir de la información contenida retornar una tabla que contenga la cantidad de personas mayores de edad según su nivel de estudios alcanzados.


In [17]:
aglomerado = input("Ingrese el NÚMERO o NOMBRE COMPLETO de aglomerado: ")

cs.tabla_nivel_educativo(data, aglomerado)

Aglomerado     2 - Gran La Plata
**************************************************************************************************************************************************************************************************************************************************************************************************************************************************************
Año     Trimestre   Primario incompleto / Ed. especial      Primario completo                       Secundario incompleto                   Secundario completo                     Superior universitario incompleto       Superior universitario completo         Sin instrucción                         
***************************************************************************************************************************************************************************************************************************************************************************************************************************

### 10- Pedir al usuario que seleccione dos aglomerados y a partir de la información contenida retornar una tabla que contenga el porcentaje de personas mayores de edad con secundario incompleto.

In [ ]:

aglo_input1 = int(input("Ingrese un aglomerado: "))
aglo_input2 = int(input("Ingrese otro aglomerado: "))

tabla_porcentaje = cs.personas_secundario_incompleto_anio_trimestre(aglo_input1, aglo_input2, data)

cs.imprimir_porcentaje_secundario_incompleto(tabla_porcentaje)

TypeError: imprimir_porcentaje_secundario_incompleto() takes 1 positional argument but 3 were given